In [10]:
import glob
import scipy
import uproot as up
import matplotlib.pyplot as plt
import awkward as ak
import numpy as np
import time
import pandas as pd
#import Pool
from multiprocessing import Pool
from scipy import stats
from scipy.stats import gaussian_kde

In [11]:
files = glob.glob('/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/*')
files


['/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/bi212',
 '/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/cf252',
 '/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/gdls_neutrons',
 '/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/na22_z700',
 '/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/po212',
 '/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/rockgamma_scint_tanks']

In [12]:
#Creates 
decay_types = ['bi212','po212','gdls_neutrons','na22_z700']
data_types = ['pulseArea_phd', 'coincidence','areaFractionTime50_ns', 
              'areaFractionTime75_ns','areaFractionTime90_ns' ,'pulseStartTime_ns', 
              'pulseEndTime_ns', 'peakAmp', 'peakTime_ns']

file_list = []

for i in decay_types:
    for n in data_types:
        file_list.extend([(i,n)])
        




In [13]:
def data_collector(decay_type, data_type):
    #List of files to be collected
    filenumber = range(100, 301 )
    totaldata = []
    #Main loop, formats the inputs to the uproot statement and appends them the total data array
    for i in filenumber:
        try:
            tfile = up.open('/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/{0}/lzap_output//LZAP_SEED{1}.root' .format(decay_type,i))
            events = tfile['Events']
            tempdata = (events['pulsesODHG.{}' .format(data_type)]).array()
            totaldata.extend(tempdata)
        except FileNotFoundError:
            pass

    finaldata = ak.flatten(totaldata)
    try:
        finaldata = ak.flatten(finaldata)
        
    except ValueError:
        pass
    return finaldata

def data_collector_mean(decay_type, data_type):
    #List of files to be collected
    filenumber = range(100, 301)
    totaldata = []
    #Main loop, formats the inputs to the uproot statement and appends them the total data array
    for i in filenumber:
        try:
            tfile = up.open('/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.11_DER_9.1.0_LZAP_5.2.8/{0}/lzap_output//LZAP_SEED{1}.root' .format(decay_type,i))
            events = tfile['Events']
            tempdata = (events['pulsesODHG.{}' .format(data_type)]).array()#
            tempdata = np.mean(tempdata)
            totaldata.append(tempdata)
        except FileNotFoundError:
            pass
                     
    return totaldata

def remove_outliers(df):
    z_scores = stats.zscore(df)

    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    new_df = df[filtered_entries]

    return new_df
        





In [14]:
t0 = time.clock_gettime(time.CLOCK_MONOTONIC_RAW)



num_processors = 30
p=Pool(processes = num_processors)
decaydata = p.starmap(data_collector, [i for i in file_list])


t1 = time.clock_gettime(time.CLOCK_MONOTONIC_RAW) - t0
print(t1)



309.3255774520221


In [15]:
display(decaydata)

[<Array [351, 314, 231, 354, ... 278, 302, 212] type='19889 * float64'>,
 <Array [107, 108, 101, 109, ... 100, 110, 94] type='19889 * int64'>,
 <Array [100, 100, 100, 100, ... 100, 100, 100] type='19889 * int64'>,
 <Array [140, 140, 140, 140, ... 150, 140, 140] type='19889 * int64'>,
 <Array [180, 180, 200, 190, ... 190, 190, 200] type='19889 * int64'>,
 <Array [1020, 1020, 1020, ... 1020, 1020, 1020] type='19889 * int64'>,
 <Array [1490, 1480, 1440, ... 1420, 1460, 1430] type='19889 * int64'>,
 <Array [3.14, 2.84, 1.78, ... 2.4, 2.78, 1.76] type='19889 * float64'>,
 <Array [60, 60, 60, 60, 60, ... 70, 70, 70, 60] type='19889 * int64'>,
 <Array [103, 74.4, 76.2, ... 93.4, 86.6, 77] type='19963 * float64'>,
 <Array [72, 57, 58, 57, 78, ... 44, 65, 60, 60] type='19963 * int64'>,
 <Array [100, 110, 100, 110, ... 100, 100, 100] type='19963 * int64'>,
 <Array [150, 160, 150, 160, ... 140, 140, 140] type='19963 * int64'>,
 <Array [200, 220, 190, 200, ... 190, 180, 180] type='19963 * int64'>,

In [20]:
def convert_to_2d(data):
    
    matrix = []
    for i in data:
        matrix.append(i)
    
    matrix = np.asarray(matrix, dtype=object)
    
    return matrix

bi212_data = convert_to_2d(decaydata[0:9])
po212_data = convert_to_2d(decaydata[9:18])
neutron_data = convert_to_2d(decaydata[18:27])
na22_data = convert_to_2d(decaydata[27:36])

bi212 = pd.DataFrame(data=bi212_data.T, columns=data_types)
po212 = pd.DataFrame(data=po212_data.T, columns=data_types )
neutron = pd.DataFrame(data=neutron_data.T, columns=data_types )
na22 = pd.DataFrame(data=na22_data.T, columns=data_types)

bi212['af50pa'] = bi212.apply(lambda row: row.areaFractionTime50_ns/row.pulseArea_phd, axis=1)
po212['af50pa'] = po212.apply(lambda row: row.areaFractionTime50_ns/row.pulseArea_phd, axis=1)
neutron['af50pa'] = neutron.apply(lambda row: row.areaFractionTime50_ns/row.pulseArea_phd, axis=1)
na22['af50pa'] = na22.apply(lambda row: row.areaFractionTime50_ns/row.pulseArea_phd, axis=1)

bi212['af75pa'] = bi212.apply(lambda row: row.areaFractionTime75_ns/row.pulseArea_phd, axis=1)
po212['af75pa'] = po212.apply(lambda row: row.areaFractionTime75_ns/row.pulseArea_phd, axis=1)
neutron['af75pa'] = neutron.apply(lambda row: row.areaFractionTime75_ns/row.pulseArea_phd, axis=1)
na22['af75pa'] = na22.apply(lambda row: row.areaFractionTime75_ns/row.pulseArea_phd, axis=1)

bi212['af90pa'] = bi212.apply(lambda row: row.areaFractionTime90_ns/row.pulseArea_phd, axis=1)
po212['af90pa'] = po212.apply(lambda row: row.areaFractionTime90_ns/row.pulseArea_phd, axis=1)
neutron['af90pa'] = neutron.apply(lambda row: row.areaFractionTime90_ns/row.pulseArea_phd, axis=1)
na22['af90pa'] = na22.apply(lambda row: row.areaFractionTime90_ns/row.pulseArea_phd, axis=1)

bi212['pulselength'] = bi212.apply(lambda row: row.pulseEndTime_ns - row.pulseStartTime_ns, axis=1)
po212['pulselength'] = po212.apply(lambda row: row.pulseEndTime_ns - row.pulseStartTime_ns, axis=1)
neutron['pulselength'] = neutron.apply(lambda row: row.pulseEndTime_ns - row.pulseStartTime_ns, axis=1)
na22['pulselength'] = na22.apply(lambda row: row.pulseEndTime_ns - row.pulseStartTime_ns, axis=1)

bi212['Tag'] = 'bi214'
po212['Tag'] = 'po214'
neutron['Tag'] = 'cf252'
na22['Tag'] = 'na22'


po212 = po212.drop(po212[po212.pulseArea_phd > 200].index)

In [21]:
bi212.to_csv('bi214_data.csv')
po212.to_csv('po214_data.csv')
neutron.to_csv('cf252_data.csv')
na22.to_csv('na22_data.csv')

In [ ]:
def data_decay(file):
    
    tfile= up.open(file)
    events=tfile["Events"]
    pulsearea=np.array(events["pulsesODHG.pulseArea_phd"])
    pulsearea=pulsearea.flatten()
    af75=np.array(events["pulsesODHG.areaFractionTime75_ns"])
    af75=af75.flatten()
    pulsestart=np.array(events["pulsesODHG.pulseStartTime_ns"])
    pulsestart=pulsestart.flatten()
    pulseend=np.array(events["pulsesODHG.pulseEndTime_ns"])
    pulseend=pulseend.flatten()
    truth = tfile['RQMCTruth']
    pp = np.array(truth['mcTruthEvent./mcTruthEvent.parentParticle'])
    pp=pp.flatten()
    
    return [pulsearea,af75,pulsestart,pulseend,pp]

def data():
    columns={'Pulse_Area' : [], 'af75' : [], 'pulsestart' : [], 'pulseend' : [], 'Tag' : []}
    files = glob.glob('/hdfs/user/ak18773/od_simulations/BACCARAT_6.2.14_DER_9.1.0_LZAP_5.4.1/od_internals/lzap_output/*')
    with mp.Pool(30) as pool:
            data = list(tqdm.tqdm(pool.imap(functools.partial(data_decay),files),total=100))
    data=pd.DataFrame(np.concatenate(data,axis=1).T.tolist(),columns=columns)
    return data


In [4]:
print('helloworld!')

helloworld!
